-----

In [ ]:
!pip install datasets
!pip install torch torchvision
!pip install transformers

---


# Siamese Algorithm for Image Comparision

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

In [31]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Preprocessing for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

In [32]:
# CNN-based feature extractor (pretrained model)
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.model = models.resnet18(pretrained=True)
        # Remove the last fully connected layer to get features
        self.model = nn.Sequential(*list(self.model.children())[:-1])

    def forward(self, x):
        return self.model(x)

In [33]:
# Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.feature_extractor = FeatureExtractor()

    def forward(self, image1, image2):
        # Extract features from both images
        feature1 = self.feature_extractor(image1)
        feature2 = self.feature_extractor(image2)

        # Flatten the feature maps
        feature1 = feature1.view(feature1.size(0), -1)
        feature2 = feature2.view(feature2.size(0), -1)

        # Calculate similarity (e.g., cosine similarity)
        similarity = F.cosine_similarity(feature1, feature2)
        return similarity

In [34]:
# Instantiate the model
model = SiameseNetwork().to(device)
model.eval()

# Function to load and preprocess images
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image.to(device)

In [35]:
# Function to compute similarity score between two images
def compute_similarity(image_path1, image_path2):
    image1 = load_image(image_path1)
    image2 = load_image(image_path2)

    with torch.no_grad():
        similarity_score = model(image1, image2)

    return similarity_score.item()

In [36]:
# Example usage:
similarity_score = compute_similarity('/content/!image.jpg', '/content/Venkata-Sai-Bhaskar-Image.jpg')
print(f'Similarity Score: {similarity_score:.4f}')


Similarity Score: 0.6734
